## Custom embeddings con Gensim

El objetivo de este notebook es utilizar documentos / corpus para crear embeddings de palabras. Se utilizará a la obra Crítica de la Razón Pura del filósofo Immanuel Kant para generar los embeddings. Se espera así que los vectores capten similitudes semánticas entre términos que son relevantes en el contexto de la obra de este pensador.

In [78]:
import string
import pandas as pd
import numpy as np

from keras.preprocessing.text import text_to_word_sequence

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

from sklearn.manifold import TSNE

import plotly.graph_objects as go
import plotly.express as px

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 1 - Carga de datos y preprocesamiento

In [79]:
krv_df = pd.read_csv('https://www.gutenberg.org/cache/epub/4280/pg4280.txt', delimiter='\t')

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones.
for _, row in krv_df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

# Renombrar la única columna del DF.
krv_df.rename(columns={ krv_df.columns[-1]: 'document' }, inplace=True)

# Explorar algunos de los tokens obtenidos.
print(f'Algunos tokens obtenidos: ', sentence_tokens[:2])

krv_df.head()

Algunos tokens obtenidos:  [['this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and'], ['most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions']]


,document
0,This ebook is for the use of anyone anywhere i...
1,most other parts of the world at no cost and w...
2,"whatsoever. You may copy it, give it away or r..."
3,of the Project Gutenberg License included with...
4,at www.gutenberg.org. If you are not located i...


### 2 - Crear los vectores (word2vec)

In [80]:
class callback(CallbackAny2Vec):
    '''
    Callback to print loss after each epoch
    '''
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        elif self.epoch % 5 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

w2v_model = Word2Vec(min_count=5,
                     window=2,
                     vector_size=300,
                     negative=20,
                     workers=1,
                     sg=1) # modelo 0:CBOW  1:skipgram

# Obtener el vocabulario con los tokens.
w2v_model.build_vocab(sentence_tokens)

# Cantidad de filas/documentos encontradas en el corpus.
print('Cantidad de documentos en el corpus:', w2v_model.corpus_count)

# Cantidad de palabras encontradas en el corpus.
print('Cantidad de palabras distintas en el corpus:', len(w2v_model.wv.index_to_key))

Cantidad de documentos en el corpus: 19487
Cantidad de palabras distintas en el corpus: 2342


### 3 - Entrenar embeddings

In [81]:
w2v_model.train(sentence_tokens,
                total_examples=w2v_model.corpus_count,
                epochs=100,
                compute_loss=True,
                callbacks=[callback()])

Loss after epoch 0: 1410867.625
Loss after epoch 5: 886133.0
Loss after epoch 10: 812294.0
Loss after epoch 15: 789826.0
Loss after epoch 20: 737610.0
Loss after epoch 25: 716918.0
Loss after epoch 30: 709100.0
Loss after epoch 35: 698432.0
Loss after epoch 40: 692574.0
Loss after epoch 45: 682076.0
Loss after epoch 50: 673384.0
Loss after epoch 55: 672116.0
Loss after epoch 60: 669444.0
Loss after epoch 65: 669588.0
Loss after epoch 70: 660712.0
Loss after epoch 75: 659900.0
Loss after epoch 80: 655916.0
Loss after epoch 85: 651424.0
Loss after epoch 90: 654356.0
Loss after epoch 95: 87768.0


(13386059, 21294000)

### 4 - Ensayar

In [82]:
def print_most_similars(model, word, topn):
    print(f'Palabras más similares a {word}:')
    for word, similarity in model.wv.most_similar(positive=[word], topn=topn):
        print(f'Palabra: {word}. Similitud: {similarity}')
    print('\n')

for word in ['space', 'time', 'reason', 'priori', 'posteriori', 'intuition', 'category', 'metaphysic', 'sensibility', 'necessary']:
    print_most_similars(w2v_model, word, 10)

Palabras más similares a space:
Palabra: time. Similitud: 0.34406179189682007
Palabra: shape. Similitud: 0.32885977625846863
Palabra: filled. Similitud: 0.3281041979789734
Palabra: ens. Similitud: 0.3260181248188019
Palabra: void. Similitud: 0.319690078496933
Palabra: contemporaneously. Similitud: 0.3093798756599426
Palabra: intuition. Similitud: 0.30840423703193665
Palabra: sensibility. Similitud: 0.30623307824134827
Palabra: spaces. Similitud: 0.3056923449039459
Palabra: antecede. Similitud: 0.30433782935142517


Palabras más similares a time:
Palabra: space. Similitud: 0.3440617322921753
Palabra: fills. Similitud: 0.3110807538032532
Palabra: unceasing. Similitud: 0.3078629672527313
Palabra: succeeding. Similitud: 0.3045356869697571
Palabra: intellect. Similitud: 0.3015232980251312
Palabra: empiricism. Similitud: 0.2970053553581238
Palabra: same. Similitud: 0.29621005058288574
Palabra: intuited. Similitud: 0.2952442765235901
Palabra: located. Similitud: 0.2916404604911804
Palabra: pa

In [83]:
def print_least_similars(model, word, topn):
    print(f'Palabras menos similares a {word}:')
    for word, similarity in model.wv.most_similar(negative=[word], topn=topn):
        print(f'Palabra: {word}. Similitud: {similarity}')
    print('\n')

for word in ['space', 'time', 'reason', 'priori', 'posteriori', 'intuition', 'category', 'metaphysic', 'sensibility', 'necessary']:
    print_least_similars(w2v_model, word, 10)

Palabras menos similares a space:
Palabra: procedure. Similitud: 0.06896268576383591
Palabra: employed. Similitud: 0.04209689050912857
Palabra: categorical. Similitud: 0.0312868170440197
Palabra: logicians. Similitud: 0.021014627069234848
Palabra: philosophical. Similitud: 0.01958725042641163
Palabra: cogitates. Similitud: 0.01919943280518055
Palabra: man. Similitud: 0.013142800889909267
Palabra: against. Similitud: 0.012682957574725151
Palabra: added. Similitud: 0.012305451557040215
Palabra: extending. Similitud: 0.012185166589915752


Palabras menos similares a time:
Palabra: help. Similitud: 0.03095529042184353
Palabra: requirement. Similitud: 0.025061752647161484
Palabra: hypotheses. Similitud: 0.02455754764378071
Palabra: essential. Similitud: 0.019175544381141663
Palabra: perfect. Similitud: 0.01153555791825056
Palabra: establishes. Similitud: 0.009404866024851799
Palabra: comparison. Similitud: 0.009064728394150734
Palabra: introduction. Similitud: 0.004550703801214695
Palabra: 

### 5 - Visualizar agrupación de vectores

In [84]:
def reduce_dimensions(model, num_dimensions=2):
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

# Graficar en 2D.
vecs, labels = reduce_dimensions(w2v_model)
MAX_WORDS = 100
fig = px.scatter(x=vecs[:MAX_WORDS, 0], y=vecs[:MAX_WORDS, 1], text=labels[:MAX_WORDS])
fig.show(renderer='colab')

El gráfico da una idea intuitiva de cómo "space", por ejemplo, está relativamente cerca de "intuition" y de "experience", pero relativamente lejos de "reason" o de "idea". (Ver comentarios al final del notebook).

In [85]:
# Graficar en 3D.
vecs, labels = reduce_dimensions(w2v_model, 3)
fig = px.scatter_3d(x=vecs[:MAX_WORDS ,0], y=vecs[:MAX_WORDS, 1], z=vecs[:MAX_WORDS, 2], text=labels[:MAX_WORDS])
fig.update_traces(marker_size=2)
fig.show(renderer='colab')

In [86]:
# Salvar vectores como .tsv para poder visualizarlos en herramientas como http://projector.tensorflow.org/.
vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)
np.savetxt('vectors.tsv', vectors, delimiter='\t')
with open('labels.tsv', 'w') as fp:
    for item in labels:
        fp.write('%s\n' % item)

### 6 - Preprocesamiento con NLTK

In [87]:
krv_str = ''
for _, row in krv_df[:None].iterrows():
    krv_str += row[0]

# Imprimir el texto completo antes del preprocesamiento.
print('Crítica de la Razón Pura antes del preprocesamiento:', krv_str)

stop_words = set(stopwords.words('english'))
sentences = sent_tokenize(krv_str.lower())
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
lemmatized_sentences = []
for sentence in sentences:
    tokens = tokenizer.tokenize(sentence.lower())
    filtered_tokens = [token for token in tokens if token not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    lemmatized_sentence = ' '.join(lemmatized_tokens)
    lemmatized_sentences.append(lemmatized_sentence)

# Construir un dataframe con el texto preprocesado.
krv_df = pd.DataFrame(lemmatized_sentences, columns=['document'])

# Imprimir el texto completo después del preprocesamiento.
print('Crítica de la Razón Pura después del preprocesamiento:', ' '.join(krv_df['document']))

Crítica de la Razón Pura antes del preprocesamiento: This ebook is for the use of anyone anywhere in the United States andmost other parts of the world at no cost and with almost no restrictionswhatsoever. You may copy it, give it away or re-use it under the termsof the Project Gutenberg License included with this ebook or onlineat www.gutenberg.org. If you are not located in the United States,you will have to check the laws of the country where you are locatedbefore using this eBook.Title: The Critique of Pure ReasonAuthor: Immanuel KantTranslator: J. M. D. MeiklejohnRelease date: July 1, 2003 [eBook #4280]                Most recently updated: July 26, 2021Language: EnglishCredits: Charles Aldarondo and David Widger*** START OF THE PROJECT GUTENBERG EBOOK THE CRITIQUE OF PURE REASON ***[Illustration]The Critique of Pure ReasonBy Immanuel KantTranslated by J. M. D. MeiklejohnContents Preface to the First Edition (1781) Preface to the Second Edition (1787) Introduction I. Of the differ

In [88]:

sentence_tokens = [sentence.split() for sentence in lemmatized_sentences]

# Explorar algunos de los tokens obtenidos.
print(f'Algunos tokens obtenidos: ', sentence_tokens[:10])

# Explorar el DF generado.
krv_df.head()

Algunos tokens obtenidos:  [['ebook', 'use', 'anyone', 'anywhere', 'united', 'state', 'andmost', 'part', 'world', 'cost', 'almost', 'restrictionswhatsoever'], ['may', 'copy', 'give', 'away', 'use', 'termsof', 'project', 'gutenberg', 'license', 'included', 'ebook', 'onlineat', 'www', 'gutenberg', 'org'], ['located', 'united', 'state', 'check', 'law', 'country', 'locatedbefore', 'using', 'ebook', 'title', 'critique', 'pure', 'reasonauthor', 'immanuel', 'kanttranslator', 'j', 'meiklejohnrelease', 'date', 'july', '1', '2003', 'ebook', '4280', 'recently', 'updated', 'july', '26', '2021language', 'englishcredits', 'charles', 'aldarondo', 'david', 'widger', 'start', 'project', 'gutenberg', 'ebook', 'critique', 'pure', 'reason', 'illustration', 'critique', 'pure', 'reasonby', 'immanuel', 'kanttranslated', 'j', 'meiklejohncontents', 'preface', 'first', 'edition', '1781', 'preface', 'second', 'edition', '1787', 'introduction', 'difference', 'pure', 'empirical', 'knowledge', 'ii'], ['human', 'int

,document
0,ebook use anyone anywhere united state andmost...
1,may copy give away use termsof project gutenbe...
2,located united state check law country located...
3,human intellect even unphilosophical state pos...
4,iii


In [89]:
w2v_model = Word2Vec(min_count=5,
                     window=2,
                     vector_size=300,
                     negative=20,
                     workers=1,
                     sg=1) # modelo 0:CBOW  1:skipgram

# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

# Cantidad de filas/documentos encontradas en el corpus.
print('Cantidad de documentos en el corpus:', w2v_model.corpus_count)

# Cantidad de palabras encontradas en el corpus.
print('Cantidad de palabras distintas en el corpus:', len(w2v_model.wv.index_to_key))

Cantidad de documentos en el corpus: 4597
Cantidad de palabras distintas en el corpus: 2244


In [90]:
lemmatized_sentences_tokenized = [sentence.split() for sentence in lemmatized_sentences]

w2v_model.train(sentence_tokens,
                total_examples=w2v_model.corpus_count,
                epochs=100,
                compute_loss=True,
                callbacks=[callback()])

Loss after epoch 0: 1031883.6875
Loss after epoch 5: 570848.75
Loss after epoch 10: 507934.5
Loss after epoch 15: 452442.0
Loss after epoch 20: 427216.0
Loss after epoch 25: 411965.0
Loss after epoch 30: 401054.0
Loss after epoch 35: 354534.0
Loss after epoch 40: 347492.0
Loss after epoch 45: 343866.0
Loss after epoch 50: 337728.0
Loss after epoch 55: 333928.0
Loss after epoch 60: 330476.0
Loss after epoch 65: 326442.0
Loss after epoch 70: 323490.0
Loss after epoch 75: 320692.0
Loss after epoch 80: 316272.0
Loss after epoch 85: 270346.0
Loss after epoch 90: 261612.0
Loss after epoch 95: 258544.0


(6984507, 9865400)

In [91]:
for word in ['space', 'time', 'reason', 'priori', 'posteriori', 'intuition', 'category', 'metaphysic', 'sensibility', 'necessary']:
    print_most_similars(w2v_model, word, 10)

Palabras más similares a space:
Palabra: filled. Similitud: 0.40263715386390686
Palabra: spaceand. Similitud: 0.36557766795158386
Palabra: theinternal. Similitud: 0.33013850450515747
Palabra: ready. Similitud: 0.32094302773475647
Palabra: portion. Similitud: 0.30420321226119995
Palabra: assign. Similitud: 0.30116844177246094
Palabra: intime. Similitud: 0.2974775433540344
Palabra: whatsoever. Similitud: 0.2958919107913971
Palabra: ordinated. Similitud: 0.29471978545188904
Palabra: theseprinciples. Similitud: 0.2922725975513458


Palabras más similares a time:
Palabra: past. Similitud: 0.3167022168636322
Palabra: duration. Similitud: 0.30187931656837463
Palabra: filled. Similitud: 0.29426345229148865
Palabra: intellect. Similitud: 0.28991350531578064
Palabra: continuum. Similitud: 0.28987231850624084
Palabra: asynthesis. Similitud: 0.2862904369831085
Palabra: change. Similitud: 0.28369033336639404
Palabra: successive. Similitud: 0.2814454138278961
Palabra: distant. Similitud: 0.280536055

En general, la primera forma de tokenización sin haber usado NLTK, aunque más simple, dio resultados más interesantes de cara a analizar cercanía semántica entre términos filosóficos.

### 7 - Comentarios finales

- Mediante el entrenamiento de embeddings con el modelo Skip-gram, se obtuvieron representaciones vectoriales de palabras tomadas de la obra Crítica de la Razón Pura. Se planteó como objetivo el observar si los vectores densos obtenidos lograban captar la cercanía semántica que algunos términos de esta obra guardan unos con otros.
- Más allá de lo que podría considerarse esperable, como que junto con "time" aparezca "past", es más interesante notar cómo junto a aquella palabra figura "intuited" entre las más similares. Asimismo, junto a "space" están "sensibility" e "intuition".
Esto es destacable porque capta un aspecto muy específico de la filosofía kantiana: Que, según este pensador, espacio y tiempo son "intuiciones puras" o "formas puras de la sensibilidad", en lugar de entidades físicas independientes del sujeto.
- Un posible problema con los vectores obtenidos es que hay términos que, aunque con frecuencia figuran juntos en el texto, tienen significados excluyentes. Por ejemplo "priori" y "posteriori" están cercanos en el espacio obtenido, a pesar de que un conocimiento es a priori si y sólo si no es a posteriori.
- El análisis de los términos menos similares no aporta tanta información como el anterior. La mayoría de las palabras menos semejantes tienen escasa o nula relevancia filosófica.
